In [ ]:
import os, sys, glob
from tqdm import tqdm
import avstack
import avapi
import cv2
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

image_dir = '../data/ADL-Rundle-6/img1/'
det_file = '../data/ADL-Rundle-6/det/det.txt'
gt_file = '../data/ADL-Rundle-6/gt/gt.txt'

## Load Ground Truths

\<frame\>, \<id\>, \<bb_left\>, \<bb_top\>, \<bb_width\>, \<bb_height\>, \<conf\>, \<x\>, \<y\>, \<z\>


In [ ]:
# Load up the ground truths
gt_boxes = []
last_frame = -1
obj_this_frame = None
with open(gt_file, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        items = line.split(',')
        frame, ID, bb_left, bb_top, bb_width, bb_height, conf, x, y, z = map(float, items)
        if frame > last_frame:
            if obj_this_frame is not None:
                gt_boxes.append(obj_this_frame)
            obj_this_frame = []
        xmin = bb_left
        ymin = bb_top
        xmax = bb_left + bb_width
        ymax = bb_top + bb_height
        box2d = avstack.geometry.Box2D([xmin, ymin, xmax, ymax], avstack.calibration.NominalCalibration)
        obj_this_frame.append(box2d)
        last_frame = frame

In [ ]:
%matplotlib inline

def viz_image_with_boxes(i_frame, boxes):
    # Visualize some ground truths
    img1 = cv2.imread(os.path.join(image_dir, '%06d.jpg'%i_frame))
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    color = (0, 255, 0)
    for box in boxes:
        img1 = cv2.rectangle(
            img1,
            (int(box.xmin), int(box.ymin)),
            (int(box.xmax), int(box.ymax)),
            color,
            2,
        )

    # Show image
    plt.imshow(img1)
    plt.show()

# Run it
i_frame = 380
viz_image_with_boxes(i_frame, gt_boxes[i_frame-1])

## Run Algorithms

In [ ]:
framerate = 30.0
detector = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='fasterrcnn', dataset='coco-person', threshold=0.5)
tracker = avstack.modules.tracking.tracker2d.SortTracker2D(framerate)

In [ ]:
all_dets = {}
all_tracks = {}
image_files = sorted(glob.glob(os.path.join(image_dir, '*.jpg')))
i_frame_start = 390
n_frames = min(len(image_files), 50)
cvt_to_rgb = True

for frame, image_file in tqdm(enumerate(image_files[i_frame_start:i_frame_start+n_frames]), total=n_frames):
    frame += i_frame_start
    # -- load image in avstack standard format
    image_data = cv2.imread(image_file)
#     if cvt_to_rgb:
#     image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)

    timestamp = i_frame/framerate
    image = avstack.sensors.ImageData(timestamp=timestamp,
                                      frame=frame,
                                      source_ID=0,
                                      source_name='camera-0',
                                      data=image_data,
                                      calibration=avstack.calibration.NominalCalibration)
    
    # -- run detections and tracking
    dets = detector(image, is_rgb=cvt_to_rgb)
    tracks = tracker(dets, frame=frame, t=timestamp)
    
    # -- save results
    all_dets[frame] = dets
    all_tracks[frame] = tracks

In [ ]:
for i in range(1, 40):
    i_frame = i_frame_start + i
    viz_image_with_boxes(i_frame, [d.box for d in all_dets[i_frame-1]])

In [ ]:
for i in range(1, 40):
    i_frame = i_frame_start + i
    viz_image_with_boxes(i_frame, [d.box for d in all_tracks[i_frame-1]])
    print(len(all_tracks[i-frame - 1]))

In [ ]:
del detector

In [ ]:
for i in range(1, 40):
    i_frame = i_frame_start + i
#     viz_image_with_boxes(i_frame, [d.box for d in all_tracks[i_frame-1]])
    print(f'Frame {i_frame:3d}, {len(all_dets[i_frame]):3d} dets,  {len(all_tracks[i_frame]):3d} tracks')

In [ ]:
# ground truth vs real detections vs tracks
i_frame = 30 + i_frame_start
viz_image_with_boxes(i_frame, gt_boxes[i_frame-1])
viz_image_with_boxes(i_frame, [d.box for d in all_dets[i_frame-1]])
viz_image_with_boxes(i_frame, [t.box for t in all_tracks[i_frame-1]])